## Filozofie projektu: Nenavist stimuluje kreativitu.

<font size="3">Tento projekt se zabyva lidmi, kteri komentuji filmu na strance CSFD.cz. CSFD je ceska obdoba stranky IMDB. 
Kazdemu registrovanemu uzivateli umoznuje ohodnotit nebo okomentovat film. Leden 2020 - 477 000 uzivatelu, 4 092 000 komentaru. CSFD umoznuje ohodnotit film jedna az peti hvezdickami. Pokud tohle nestaci, je mozne pouzit hodnoceni "odpad!", ktere byly inspiraci k vytvoreni tohoto projektu. 

Pri prohlizeni filmu s dobrymi hodnocenimi me napadlo podivat se na lidi, kteri tyto filmy nesnaseji ("odpad"). Ukazalo se, ze potreba ukazat vsem, jak moc se ve svem minieni o danem filmu pletou, vyvolava v techto lidech znacnou kreativitu pri zanechani negativniho komentare. U nekterych se clovek muze dobre zasmat ryzi zlosti autora:


_"Tvoji nenávist ke značce vody ti uvěřím, až **vymrdáš** hrdlo láhve, s tím svým tenkým **kokotem** to bude Mattoni malina. Jistě to potají chlastáš s **buzeranty**, se kterými se slézáš v barech, ty **píčo** s maskou. S tím už nepostrašíš ani dítě z mateřské školy, **pičo** ukřivděná, ubrečená, tupá. Táhni do **piče** s tou tvojí mentálně ubohou hudbou! 0%"_ - kingik

Cilem je nalezt nastvane ... zejmena u dobrych filmu.

Komentare maji sve vlastni skore, zalozene na nenavisti textu komentare. Skore se pricita za sproste slova, pouziti CAPS LOCKU a opakovani stejneho znaku ("fuuuuuuuj!!!!"). 

Krome komentaru s hodnocenim "odpad!" pracuje projekt take s hodnocenim jedne hvezdicky. Casto se v takych komentarich vyskytuje fraze: _"Jedna hvezdicka za (insert herce nebo scenu). Jinak Odpad!"_ </font>

In [1]:
import urllib
import numpy as np
import pandas as pd
from collections import Counter
from tqdm.notebook import tqdm
from datetime import datetime
from time import time
from functools import wraps
import re
import pickle
import line_profiler
# %load_ext line_profiler

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

from IPython.display import Markdown, display

In [24]:
# def reload_driver():
#     global driver
#     options = Options()
#     options.headless = True
#     options.set_preference("javascript.enabled", False)
    
#     driver = webdriver.Firefox(options=options)
#     driver.set_page_load_timeout(100)
#     return driver

In [27]:
def reload_driver():
    global driver
    caps = DesiredCapabilities().FIREFOX
#     caps["pageLoadStrategy"] = "normal"  #  complete
    options = Options()
#     options.headless = True
    options.set_preference("javascript.enabled", False)
    caps["pageLoadStrategy"] = "eager"
    driver = webdriver.Firefox(desired_capabilities=caps, options=options)
    
    return driver

In [31]:
reload_driver()

<selenium.webdriver.firefox.webdriver.WebDriver (session="406102a8-6932-47a8-a968-604a84261dc9")>

In [39]:
# %%writefile simulation.py
# co kdyz csfd odmitne request
# hodnoceni s komentarem je zahrnoto v hodnocenich bez komentaru
# nechapu .. hodnoceni, videli ... stejny film, hvezdicku ... za -- regex

RATING_SYSTEM = {"film c0": "white",
                 "film c1": "red",
                 "film c2": "blue",
                 "film c3": "black"}

EVIL_WORDS = {"sračk", 'stupidní', 'kretén', "kokot", "zkurvenec", 'hovno', 'hovna', 'hoven', 'čurák', 'hajzl',
              'kurv', 'po*rať', 'prdel', 'propagand', 'idiot', 'buzny', 'buzna', 'mrtka', 'píčo',
              'blití', 'fuj', 'sra*ka', 'posrat', 'demenci', 'pí*ovina', 'píčovina', 'kravin', 'kripl',
              'vymrdaných', 'shit', 'stád', 'pí..vina', 'dement', 'zasvin', 'prase', 'úch*l', 'poser',
              'blb', 'magor', 'magoř', 'sračku', 'mrd*ka', 'držka', 'debil', 'retard', 'neger','hovně',
             'pi*ovina', 'kund', 'piče', 'piči', 'mrd', 'pičo', 'píčo', 'píče', 'negr','sran', 'buzerant',
             'ošukán', 'posral', 'čuráci', 'serete', 'šukat', 'pošuk', 'pošuc', 'šukání', 'fuck', 'debíl',
             'piiicovinu', 'piiico', 'hnus', 'ho*no', 'hnus', "hovnům", 'zmrd', 'pičus', 'srán', 'h..vna',
             's..re', 'm..dka', 'p..ča'}

MISTAKE_WORDS = {'knihovna', 'knihovny', 'filozofující', 'sranda', 'srandu', 'srandy', 'sekund','sekunda', 'sekundy',
                'duchovna', 'duchovno'}

class Comment:
    def __init__(self, rating, text, approximate_movie_rating, movie_name, date=None,
                 time=None, author=None, movie_rating=None):
        
        self.rating = rating
        self.text = text
        self.approximate_movie_rating = approximate_movie_rating
        self.movie_rating = movie_rating
        self.movie_name = movie_name
        self.date = date
        self.time = time
        self.author = author
        self.score = self.evil_score()
        
    def add_author(self, author):
        self.author = author
        
    def evil_score(self):
        score = 0
        for word in self.text.split():
            for EVIL in EVIL_WORDS:
                if EVIL in word.lower() and word.lower() not in MISTAKE_WORDS:
                    score += 2
                    
            repeated_char, number_of_repeats = maxRepeating(word)
            if number_of_repeats > 2: # 0,33 zvoleno jen tak for fun
                number_of_repeats = 5 if number_of_repeats > 5 else number_of_repeats
                score += 0.33 * number_of_repeats
                if repeated_char == "!":
                    score += 0.33 * number_of_repeats
                
            if caps_lock(word):
                score += 0.5
        
        return round(score,1)

    def to_dict(self): # pro pandas, kdyby nahodou
        return {
            'author': self.author,
            'rating': self.rating,
            'text': self.text,
            'hate_score': self.score,
            'movie_name': self.movie_name,
            'movie_rating_apr': self.approximate_movie_rating,
            'date': self.date,
            'time': self.date
        }
    
    def __repr__(self):
        return '{}, movie name: {}, skore komentare: {}, datum a cas: {} {}'.format(self.author,
                                                                                 self.movie_name,
                                                                                 self.score,
                                                                                 self.date,
                                                                                 self.time)


def timing(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        start = time()
        result = f(*args, **kwargs)
        end = time()
        print('Elapsed time: {}'.format(end-start))
        return result
    return wrapper


def printmd(string):
    display(Markdown(string))

        
def get_best_movie_score(driver):
    """
    Vetsina filmu nema skore nejlepsich filmu
    """
    try:
        return driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[1]/p/a[1]')
    except:
        return None

    
def get_favourite_movie_score(driver):
    """
    Vetsina filmu nema skore oblibenosti
    """
    try:
        return driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[1]/p/a[2]')  
    except:
        return None    

    
def get_full_czech_name(driver):
    return driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/h1')


def get_comment_text(comment):
    return comment.find_element_by_class_name('post').text


def get_comment_author(comment):
    return comment.find_element_by_class_name('author')


def get_czech_name_in_comment(comment):
    return comment.find_element_by_class_name('author').text


def get_comment_rating(comment):
    """
    return None pro komentar bez hodnoceni
           0 pro komentar typu odpad!
           pocet hvezdicek jinak
    """
    try:
        author_rating = comment.find_element_by_class_name('rating').get_attribute('alt')
    except:
        return None
    
    if not author_rating:
        return 0
    
    return len(author_rating)
   
    
def remove_last_line_from_string(s):
    return s[:s.rfind('\n')]


def count_in_string(string, char):
    return strin.count(char)


def movie_rating_to_stars(rating):
    if rating == 0:
        return "odpad!"
    else:
        return rating * "*"
    

def get_approx_movie_rating_from_comment(comment):
    """ Z komentare v profilu nalezne priblizne hodnoceni filmu podle barvy ikony
    
    Parameters:
        comment(webdriver.element?): comment inside users profile
    
    Returns:
        string:color of movie icon
    
    """
    movie_rating = comment.find_element_by_class_name("film")
    return RATING_SYSTEM[movie_rating.get_attribute("class")]



def maxRepeating(string): 
    """ Najde znak, ktery se nejvicekrat vyskytuje za sebou ve stringu 
    Return: tuple (znak, pocet opakovani)
    """
  
    n = len(string) 
    count = 0
    res = string[0] 
    cur_count = 1
  
    for i in range(n):
        if (i < n - 1 and string[i] == string[i + 1]): 
            cur_count += 1
  
        else: 
            if cur_count > count and string[i] != ".": 
                count = cur_count 
                res = string[i] 
            cur_count = 1
    return res, count


def caps_lock(string):
    matched = re.match('^[a-zA-Z]+(\!*\?*\.*\,*)?$', string) # isupper() nestaci, :-D vraci True
    
    if matched is None:
        return False
    
    if len(string) > 2 and string.isupper():
        return True
    
    else:
        return False

    
def comment_split(comment_text): # pouzite jen k vytvoreni counteru slov
    words = []
    for word in comment_text.split():
        if word[-1] == "," or word[-1] == ".":
            word = word[0:-1]
            
        word = word.lower()
        words.append(word)
        
    return words
    
    
def replace_in_comment(comment_text, replace): # vic testovani!!
    words = comment_text.split()
    for i,word in enumerate(words):
        for EVIL in EVIL_WORDS:
            if EVIL in word.lower() and word.lower() not in MISTAKE_WORDS:
                words[i] = replace + word + replace
    return " ".join(words)

In [4]:
class Profile:
    def __init__(self, link, num_of_ratings, comments, score):
        self.link = link
        self.id = self.find_id()
        self.num_of_ratings = num_of_ratings
        self.comments = comments
        self.num_of_comments = len(self.comments)
        self.score = round(score,1)
        self.last_updated = datetime.today().strftime("%d-%m-%Y")
        
    def find_id(self):
        id_re = re.match(".*/uzivatel/(\d+)", self.link)
        try:
            aids = int(id_re.group(1))
            return aids
        except:
            raise ValueError("TOHLE BY SE NEMELO STAT U NORMALNIHO URL")
            
    def __repr__(self):
        return "{}, Pocet komentaru: {}, skore: {}".format(self.link, self.num_of_comments, self.score)
    
    def to_dict(self): # pro pandas
        return {
            'id': self.id,
            'num_of_comments': self.num_of_comments,
            'score' : self.score
        }
            
# profile_comments('https://www.csfd.cz/uzivatel/363-woody', driver)

def profile_comments(driver):
    """ Nacte vsechny
    """
    current = driver.current_url
    try:
        driver.get(current + 'komentare/podle-rating/')
        
    except TimeoutException as e:
        print(e)
        driver.execute_script("window.stop();")
    
    try:
        pages = driver.find_element_by_xpath('/html/body/div[2]/div[3]/div[2]/div[1]/div[3]/div[3]').text
    
    except NoSuchElementException: # pokud ma pouze jednu stranku komentaru
        comment_objects = profile_comments_on_page(driver, current + 'komentare/podle-rating/')
        comment_objects = [_ for _ in comment_objects if _ is not None]
        list(map(lambda c:c.add_author(current), comment_objects))
        return comment_objects
    
    pages = re.findall(r'\d+', pages)[-1] # posledni cislo v seznamu, potrebuje poradne otestovat(vypada OK)
    try:
        pages = int(pages)
    except:
        raise ValueError("Neco divneho s typem")
        
    next_run = True
    comment_objects = []
    while next_run:
        if pages < 0:
            break
        # muze poslat -1 do url .../strana--1/
        local_comments = profile_comments_on_page(driver, current + 'komentare/podle-rating/strana-' + str(pages))
        if len(local_comments) == 0: # pokud nejsou zadne komentare s 0 nebo 1 hvezdou
            next_run = False
            break
            
        local_comments = [c for c in local_comments if c is not None]
        list(map(lambda c:c.add_author(current), local_comments))
        
        comment_objects = comment_objects + local_comments
        pages -= 1 
        
    return [c for c in comment_objects if c is not None] # nevim proc


def profile_comments_on_page(driver, url):
    """ Nacte komentare na strance s cislovanim
    """
    driver.get(url)
#     comments = driver.find_elements_by_class_name("post")
    
    item_list = driver.find_element_by_class_name("ui-posts-action-list")
    items = item_list.find_elements_by_tag_name("li")
    
    out = []
    for item in items[::2]:
        
        comment = item.find_element_by_class_name('post')
        comment_rating = get_comment_rating(comment)
        
        if comment_rating is None:
            out.append(None) # HACK
            continue
            
        if comment_rating >1:
            continue
        
        movie_approx = get_approx_movie_rating_from_comment(comment)
        if movie_approx != 'red':
            continue
        
        date = item.find_element_by_class_name('date')
        splited = date.text.split()
        time = splited[1]
        date = splited[0]
        
        comment_text = comment.text.split('\n')[1:][0] # hack, prvni radek neni text komentare
        comment_text = comment_text.replace("*", "#") # nahrazeni hvezdicek, ktere se v komentari vyskytuji
        
        movie_name = get_czech_name_in_comment(comment)
        
        comment_object = Comment(comment_rating, comment_text, movie_approx, movie_name, date=date, time=time)
        out.append(comment_object)
        
    return out # predelat na yield


def profile_ratings(driver): # zatim nepouzito, trva hrozne dlouho
    
    current = driver.current_url
    driver.get(current + '/hodnoceni/')
    
    ratings = driver.find_elements_by_class_name('rating')
    
    all_ratings = []
    for rating in ratings:
        rating = rating.get_attribute('alt')
        if not rating:
            all_ratings.append(0)
        else:
            all_ratings.append(len(rating))
#         print(rating.get_attribute('alt'))
    
    next_run = True
    i = 2
    print("Prochazim stranky s hodnocenim")
    while next_run:
        driver.get(current + 'hodnoceni/strana-' + str(i) + '/')
        ratings = driver.find_elements_by_class_name('rating')
        i += 1
        
        if len(ratings) == 0: # prazdna stranka
            next_run = False
            continue
        
        
        for rating in ratings:
            rating = rating.get_attribute('alt')
            if not rating:
                all_ratings.append(0)
            else:
                all_ratings.append(len(rating))

    return all_ratings


def print_profile(profile, score_limit):
    print("Profil:", profile.link)
    print("Naposledy ulozeno:", profile.last_updated)
    print("Nenavistne komentare u filmu/serialu s vetsim hodnocenim nez 70%")
    print()
    
    evil_comments = Counter()
    sum_of_comment_score = 0
    comments = profile.comments
    comments.sort(key=lambda x: x.score, reverse=True)
    
    for comment in comments:
        if comment.approximate_movie_rating == "red": # pokud hodnoti cerveny film odpadem nebo 1 hvezdou - melo by jit bez if
            if comment.score < score_limit:
                continue
            
            comment_text = comment.text
            
            comment_text = replace_in_comment(comment_text, '**')
            comment.text = comment_text
            
            sum_of_comment_score += comment.score # # nebo tady pro spatne hodnocene
                
            print(comment.movie_name, movie_rating_to_stars(comment.rating), comment.score)
            printmd(comment_text)
            print()
                
            comment_words = comment_split(comment.text)
            for word in comment_words:
                evil_comments[word] += 1
    
    print("Skore profilu:", round(sum_of_comment_score,1))
    print("Pomer unikatnich a vsech slov", len(evil_comments)/sum(evil_comments.values()))


# @timing
def profile_summary(driver,url, print_command=False): # co kdyz nekdo zada platnou adresu ale nebude tam profil ... checknout url
    
    driver.get(url) # vykaslat se na hlavni stranku profilu, muze to trvat dlouho, maji tam fotky, gify atd.
    
    try:
        error = driver.find_element_by_id('pg-web-error') # nejake divne 
        if error:
#             print(error.text)
            raise ValueError("Neplatna url adresa")
    except NoSuchElementException:
        pass
    
    comments = profile_comments(driver)
    
    evil_comments = Counter()
    sum_of_comment_score = 0
    for comment in comments:
        
#         sum_of_comment_score += comment.score # # bud tady pro vsechny filmy
        if comment.approximate_movie_rating == "red": # pokud hodnoti cerveny film odpadem nebo 1 hvezdou - melo by jit bez if
            
            comment_text = replace_in_comment(comment.text, '**')
            comment.text = comment_text
            sum_of_comment_score += comment.score  # # nebo tady pro spatne hodnocene
                
            comment_words = comment_split(comment.text)
            for word in comment_words:
                evil_comments[word] += 1
        
    profile_object = Profile(url, 0, comments, sum_of_comment_score)
    return profile_object

In [48]:
# profil = profile_summary(driver, 'https://www.csfd.cz/uzivatel/242454-maq/')
# print_profile(profil,5)

In [5]:
class Movie:
    
    def __init__(self, czech_name, rating, comments, best_movie = None, fav_movie = None):
        self.czech_name = czech_name
        self.rating = rating
        self.comments = comments
        self.best_movie = best_movie
        self.fav_movie = fav_movie
        self.hate_score = self.hate()
        
    def hate(self):
        if len(self.comments) == 0:
            return 0
        else:
            total = 0
            for comment in self.comments:
                total += comment.score
                
            return total/len(self.comments)
        
    def __repr__(self):
        return '{} Hodnoceni: {} Pocet nenavistnych komentaru: {}, Nenavist skore: {}'.format(self.czech_name, 
                                                                            self.rating,
                                                                            len(self.comments),
                                                                           round(self.hate_score, 2))

    
def get_transformed_comment(comment):
    comment = get_comment_text(comment)
    comment = remove_last_line_from_string(comment) # posledni radek je datum komentare
    comment = comment.replace("*", "#")
    comment = replace_in_comment(comment, '**')
    return comment
    

def create_comment_objects(comments, movie_name):
    comment_objects = []
    for comment in tqdm(reversed(comments)):
        rating = get_comment_rating(comment)
        
        if rating is None:
            continue
             
        # zrychli vypocet, ale prijde o schopnost vypsat counter hodnoceni
        if rating > 1:
            break
        
        trash_comment = get_transformed_comment(comment)
        author = get_comment_author(comment)
        profile_link = comment.find_element_by_css_selector(".author [href]").get_attribute('href')

        comment_object = Comment(rating, trash_comment, 'red', movie_name, author = profile_link) # FIX
        comment_objects.append(comment_object)
        
    comment_objects.sort(key=lambda x: x.score, reverse=True)
    return comment_objects
    
    
    
# @timing
def movie_summary(url, print_command=False):
    """
    finds 
    """
    
    if '/prehled/' in url:
        url = url.replace('/prehled/', '/')
    
    driver.get(url+'/komentare/podle-rating/?all=1')
    
    full_czech_name = get_full_czech_name(driver).text
    movie_rating = driver.find_element_by_class_name('average')
    
    best_movies = get_best_movie_score(driver)
    favourite_movies = get_favourite_movie_score(driver)
    
    if favourite_movies:
        favourite_movies = favourite_movies.text
    if best_movies:
        best_movies = best_movies.text
    
    comments = driver.find_elements_by_xpath('//*[starts-with(@id, "comment")]')
    comment_objects = create_comment_objects(comments, full_czech_name)
    
    movie_object = Movie(full_czech_name, movie_rating.text, comment_objects, best_movies, favourite_movies)
    
    if print_command:
        print(full_czech_name)
        print("Hodnoceni filmu: {}".format(movie_rating.text)) # co kdyz zatim nema rating
#         print("Hodnoceni pouze z komentaru: {}%".format(int(round(np.mean(avg)*20,0)))) # hodnoceni lidi s komentari muze byt rozdilne od vsech hodnoceni
        print()
        if favourite_movies:
            print(favourite_movies)
        if best_movies:
            print(best_movies)
            
        print('Pocet komentaru: {}'.format(len(comments)))
        print('Pocet komentaru odpad nebo jedna *: {}'.format(len(comment_objects)))
        print()
                
            
    return movie_object
    

## Vytahovani informaci ze zebricku

In [6]:
class leaderboard_item:
    
    def __init__(self, order, name, link, rating, number_of_ratings):
        self.order = order
        self.name = name
        self.link = link
        self.rating = rating
        self.number_of_ratings = number_of_ratings
        
    def __repr__(self):
        return  '{} {}, {}, {}'.format(self.order, self.name, self.rating, self.number_of_ratings)

def csfd_leaderboards(type_of_board): 
    
    url = ('https://www.csfd.cz/zebricky/' + type_of_board + '?show=complete')
    driver.get(url)
    items = driver.find_elements_by_xpath('/html/body/div[2]/div[3]/div[2]/table/tbody/tr')
    
    leaderboard_items = []
    for item in tqdm(items):
        
        try:
            order = item.find_element_by_class_name('order').text
            name = item.find_element_by_class_name('film').text
            link = item.find_element_by_css_selector(".film [href]").get_attribute('href')
            rating = item.find_element_by_class_name('average').text
            number_of_ratings = item.find_element_by_class_name('count').text
            
            board_item = leaderboard_item(order, name, link, rating, number_of_ratings)
            leaderboard_items.append(board_item)
            
        except NoSuchElementException: # pravdepodobne prazdny <tr> element
            continue
            
    return leaderboard_items


# leaders = csfd_leaderboards('nejlepsi-filmy/')

## Ukladani a nacitani ze souboru

In [18]:
def get_profiles_from_urls(profile, ids):
    """ Z url adresy vrati profil objekt
    """
     # odstraneni duplikatu (udelat lip, tohle nacita cely profil a az potom checkuje)
    print(profile)
    id_re = re.match(".*/uzivatel/(\d+)", profile)
    try:
        id_re = int(id_re.group(1))
    except:
        raise ValueError("TOHLE BY SE NEMELO STAT")
        
    if id_re not in ids:
        profile = profile_summary(driver, profile)
        return profile
    else:
        return None

    

def store_id(ids):
    """ Ulozi pole idecek do souboru"""
    with open('ids.txt', 'a') as file:
        for id in ids:
            file.write("{} ".format(id))
            
def load_ids():
    """ Nacte idcka ze souboru"""
    out = []
    with open('ids.txt', 'r') as f:
        content = f.readlines()
        if content:
            for num in content[0].split():
                out.append(int(num))

    return out
# profile_objects = get_profiles_from_urls(worst_profiles)

def store_profiles(profile):
    """ ulozi profilove objekty do souboru
    
    raises ValueError"""
    if profile is None:
        raise ValueError("None misto profilu")
        
    with open('profiles.pkl', 'ab') as output:
        add_id = []
        
        ids = load_ids()
            
        if profile.id not in ids:
            pickle.dump(profile, output, pickle.HIGHEST_PROTOCOL)
            add_id.append(profile.id)
            print('ADDING: ',profile.id)
                
        store_id(add_id)
        

def load_profiles():
    """ Nacte vsechny profily ze souboru zpatky jako objekty"""
    objs = []
    with open('profiles.pkl', 'rb') as f:
        
        while 1:
            try:
                o = pickle.load(f)
            except EOFError:
                break
            objs.append(o)
    return objs
        
# load_profiles()
def add_interesting_user(url, note=None):
    with open('interesting.txt', 'a') as file:
        file.write(url)
        if note is not None:
            file.write(' ## '+ note)
        file.write('\n')

## Vysledky

In [49]:
def main():
    driver = reload_driver()
    board = leaderboard('nejoblibenejsi-serialy/') # generator
    
    for b in board:
        print(len(b.comments))
        worst_profiles = vulgar_profiles(b)
        print(len(worst_profiles))
        store_bad_profiles(worst_profiles)
# main()

In [9]:
def leaderboard(board_type):
    out = []
    leaders = csfd_leaderboards(board_type)
    
    for leader in leaders:
        yield movie_summary(leader.link)
        

def vulgar_profiles(movie):
    worst_profiles = []
    aids = load_ids()
#     for movie in movies: # poslat 1 nebo vsechny?
    comments = movie.comments
#         worst_profiles = []

    print(movie.czech_name)
    for comment in comments:
        if comment.score > 1:
            id_re = re.match(".*/uzivatel/(\d+)", comment.author)
            try:
                id_re = int(id_re.group(1))
            except:
                raise ValueError("TOHLE BY SE NEMELO STAT U NORMALNIHO URL")

            if id_re not in aids:
                print(comment.author,comment.score)
                worst_profiles.append(comment.author)
                printmd(comment.text)
#         yield worst_profiles
    print()
    return worst_profiles


def store_bad_profiles(worst_profiles):
    
    profiles_to_store = list(set(worst_profiles))
    ids = load_ids()
    
    for worst in tqdm(profiles_to_store):
        tmp = get_profiles_from_urls(worst, ids) # osetrit duplikaty ve worst, bude vracet None pokud id profilu bude v ids
        try:
            store_profiles(tmp)
        except ValueError as e:
            print(e)
            

In [50]:
def print_stored_profiles():
    movie_counter = Counter()
    stored_profiles = load_profiles()
    stored_profiles.sort(key=lambda x: x.score, reverse=True)
    for profile in stored_profiles:
        if profile.score > 1:
            print(profile.link, profile.score)
            comments = profile.comments
            for comment in comments:
                movie_counter[comment.movie_name] += 1
                
    most_common = movie_counter.most_common(50)
    for m in most_common:
        print(m[1], ':', m[0])
        
# print_stored_profiles()

In [ ]:
'knihovny', 'průser', 'Serete', 'filozofující' - fuj, citace z filmu

## Analyzy

In [107]:
stored_profiles = load_profiles()

In [110]:
def comments_to_dataframe():
    all_comments = []
    for profile in stored_profiles:
        comments = profile.comments
        all_comments = all_comments + comments
    
    frame = pd.DataFrame.from_records([comment.to_dict() for comment in all_comments if comment.rating <2 and comment.approximate_movie_rating == "red"])
#     frame = frame.sort_values(by=['hate_score'], ascending=False)
    return frame
    
    
def profiles_to_dataframe():
    frame = pd.DataFrame.from_records([profile.to_dict() for profile in stored_profiles])
    return frame

    
# profiles_to_dataframe()

In [111]:
profiles_frame = profiles_to_dataframe().sort_values(by=['score'], ascending=False)
profiles_frame

,id,num_of_comments,score
3,172466,0,1686.76
184,163977,0,956.52
89,141370,0,922.54
29,142650,0,585.68
72,26324,0,550.06
...,...,...,...
124,366041,0,0.00
131,221906,0,0.00
18,15559,0,0.00
314,63075,0,0.00


## Testovani konkretnich filmu a profilu

In [51]:
# movie = movie_summary('https://www.csfd.cz/film/228329-avatar/prehled/', print_command=True)
# c = 0
# for comment in movie.comments:
#     if comment.score > 1:
#         print(comment)
#         c +=1
        
# print(c)

In [304]:
reload_driver()

In [52]:
# profil = profile_summary(driver, 'https://www.csfd.cz/uzivatel/242454-maq/')
# print_profile(profil,5)

In [53]:
# profil2 = profile_summary(driver, 'https://www.csfd.cz/uzivatel/125208-katullka')
# print_profile(profil2,2)

In [115]:
# for comment in profil.comments:
#     print(profil.link)
#     if comment.score > 1:
#         print(comment.movie_name,comment.score)
#         printmd(comment.text)
#         print()

In [33]:
add_interesting_user('https://www.csfd.cz/uzivatel/125208-katullka', 'zatim asi jedina holka, vsichni jsou buzeranti')

## Testy metod, for fun

In [36]:
import unittest
import mock

In [41]:
class Test_util_methods(unittest.TestCase):
    
    def load(self):
#         reload_driver()
        driver.get('https://www.csfd.cz/uzivatel/334074-andy60/')
    
    def test_profile_from_url(self):
        aids = []
        result = get_profiles_from_urls('https://www.csfd.cz/uzivatel/334074-andy60/', aids)
        self.assertIsInstance(result, Profile)
#         self.assertEqual()
        
        aids = [334074]
        result = get_profiles_from_urls('https://www.csfd.cz/uzivatel/334074-andy60/', aids)
        self.assertIsNone(result)
        
        aids = []
        with self.assertRaises(ValueError):
            get_profiles_from_urls('https://www.csfd.cz/uzivatel/334074andy60/', aids)
    
    def test_replace_in_comment_simple(self):
        result = replace_in_comment("kunda", '**')
        self.assertIn('**kunda**', result)
        
        result = replace_in_comment("kkkkkunda", '**')
        self.assertIn('**kkkkkunda**', result)
        
        result = replace_in_comment("kundaaaa", '**')
        self.assertIn('**kundaaaa**', result)
        
        result = replace_in_comment('sekunda', '**')
        self.assertNotIn('**sekunda**', result)
        
    def test_replace_in_comment_longer(self):
        result = replace_in_comment("prdel plna hovna", '**')
        c = result.count("*")
        self.assertEqual(c, 8)
        
    
    def test_rating_five(self):
        result = movie_rating_to_stars(5)
        self.assertEqual(result, "*****", 'spatny pocet hvezdicek')
        
    def test_rating_zero(self):
        result = movie_rating_to_stars(0)
        self.assertEqual(result, 'odpad!', 'rating 0 znamena odpad!')
        
    def test_caps_lock(self):
        result = caps_lock('LOCK')
        self.assertTrue(result, 'Ocekavam True pro LOCK')
        
        result = caps_lock('LoCK')
        self.assertFalse(result, 'Ocekavam False pro LoCK')
        
        result = caps_lock(':-D')
        self.assertFalse(result, "Ocekavam False pro :-D")
        
        result = caps_lock('AADDD!!!!!')
        self.assertTrue(result)
        
        result = caps_lock('89-T')
        self.assertFalse(result)
        
        result = caps_lock('aadBK')
        self.assertFalse(result)
        
    def test_max_repeating(self):
        result = maxRepeating("looooock")
        self.assertEqual(result[0], 'o')
        self.assertEqual(result[1], 5)
        
        result = maxRepeating("xaxax")
        self.assertLessEqual(result[1], 1)
        
        result = maxRepeating("!!!????")
        self.assertEqual(result[0], '?')
        self.assertEqual(result[1], 4)
        
    def test_profile_comments(self):
        self.load()
        result = profile_comments(driver)
        
        
    def test_store_profile(self):
        with self.assertRaises(ValueError):
            store_profiles(None)
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...

https://www.csfd.cz/uzivatel/334074-andy60/


......

https://www.csfd.cz/uzivatel/334074-andy60/
https://www.csfd.cz/uzivatel/334074andy60/



----------------------------------------------------------------------
Ran 9 tests in 22.939s

OK


In [151]:
mock_class = mock.NonCallableMock()
mock_class.some_method.return_value = 42
print(mock_class.some_method(6, 9))
mock_class.some_method.assert_called_once_with(6, 9)

42


In [298]:
reload_driver()

In [322]:
driver.get('https://www.csfd.cz/uzivatel/311349-pipapa/komentare/podle-rating/strana-9/')